# Landsat Surface Reflectance and Surface Temperature <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">


* **Products used:** 
[ls5_c2l2](https://explorer.digitalearth.africa/ls5_c2l2),
[ls7_c2l2](https://explorer.digitalearth.africa/ls7_c2l2),
[ls8_c2l2](https://explorer.digitalearth.africa/ls8_c2l2)



## Background

The United States Geological Survey's (USGS) [Landsat satellite program](https://www.nasa.gov/mission_pages/landsat/overview/index.html) has been capturing images of the African continent for more than 30 years. These data are highly useful for land and coastal mapping studies.

DE Africa's Landsat data is ingested from the [USGS Collection 2, level 2](https://www.usgs.gov/land-resources/nli/landsat/landsat-collection-2?qt-science_support_page_related_con=1#qt-science_support_page_related_con) archive and forms a single, cohesive Analysis Ready Data (ARD) package, which allows you to analyse surface reflectance data as is without the need to apply additional corrections.

**Important details:**
* Surface reflectance product
    * Native scaling range: `1 – 65,455 (0 is no-data)`
    * To achieve surface relactance, normalise values to 0-1 using: `ds = ds * 2.75e-5 - 0.2`
* Includes surface brightness temperature
    * Temperature scaling 1 - 65,535
* [CFMask](https://github.com/USGS-EROS/espa-cloud-masking/tree/master/cfmask) used as cloud mask
* Native pixel alignment is centre
* Status: Operational
* Date-range: 1984 to Present
* Spatial resolution: 30 x 30 m

> Note: For a detailed description of DE Africa's Landsat archive, see the DE Africa's [Landsat docs]()

## Description

In this notebook we will load **Landsat** data using two methods.  Firstly, we will use [dc.load()](../Beginners_guide/03_Loading_data.ipynb) to return a time series of satellite images from a single sensor.  Secondly, we will load a time series using the [load_ard()](../Frequently_used_code/Using_load_ard.ipynb) function, which is a wrapper function around the `dc.load` module.  This function will load all the images from Landsat 5,7 & 8, combine them, and then apply a cloud mask. The returned `xarray.Dataset` will contain analysis ready images with the cloudy and invalid pixels masked out.

Topics covered include:
1. Inspecting the Landsat products and measurements available in the datacube
2. Using the native `dc.load()` function to load in Landsat data from a single satellite
3. Using the `load_ard()` wrapper function to load in a concatenated, sorted, and cloud masked time series from Landsat 5, 7 & 8.

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [ ]:
import datacube

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app="Landsat Collections")

## Available products and measurements

### List products

We can use datacube's list_products functionality to inspect the DE Africa's Landsat products that are available in the datacube. The table below shows the product names that we will use to load the data, a brief description of the data, and the satellite instrument that acquired the data.

In [ ]:
# List Landsat products available in DE Africa
dc_products = dc.list_products()
display_columns = ['name', 'description']
dc_products[dc_products.name.str.contains(
    'c2l2').fillna(
        False)][display_columns].set_index('name')

### List measurements

We can further inspect the data available for each Landsat product using datacube's list_measurements functionality. The table below lists each of the measurements available in the data.

In [ ]:
dc_measurements = dc.list_measurements()
dc_measurements.loc['ls8_c2l2']

## Load Landsat using `dc.load()`

Now that we know what products and measurements are available for the products, we can load data from the datacube using `dc.load`.

In the example below, we will load data from Landsat 8 from Cape Town for South Africa in January 2018. We will load data from six spectral satellite bands, as well as cloud masking data (`'quality_l2_aerosol'`). By specifying `output_crs='EPSG:6933'` and `resolution=(-30, 30)`, we request that datacube reproject our data to the African Equal Albers coordinate reference system (CRS), with 30 x 30 m pixels. Finally, `group_by='solar_day'` ensures that overlapping images taken within seconds of each other as the satellite passes over are combined into a single time step in the data.

> Note: For a more general discussion of how to load data using the datacube, refer to the [Introduction to loading data](../Beginners_guide/03_Loading_data.ipynb) notebook.


In [ ]:
# load data
ds = dc.load(product="ls8_c2l2",
             measurements=['red', 'green', 'blue', 'swir_1',
                           'swir_2', 'quality_l2_aerosol']
             output_crs='EPSG:6933',
             x=(-33.9, -33.95),
             y=(18.58, 18.63),
             time=("2018-01", "2018-01"),
             resolution=(-30, 30),
             group_by="solar_day",
             )

print(ds)

### Plotting Landsat data

We can plot the data we loaded using the `rgb` function. By default, the function will plot data as a true colour image using the 'red', 'green', and 'blue' bands.


In [ ]:
rgb(ds, index=0)

## Load Landsat using `load_ard`

This function will load images from all the landsat sensors, concatenate and sort the observations by time, and apply a cloud mask.
The result is an analysis ready dataset.
You can find more information on this function from the [Using load ard](../Frequently_used_code/Using_load_ard.ipynb) notebook.

In [ ]:
ds = load_ard(dc=dc,
              products=["ls8_c2l2",
                        "ls7_c2l2",
                        "ls5_c2l2"],
              measurements=['red', 'green', 'blue'],
              output_crs='EPSG:6933',
              x=(-33.9, -33.95),
              y=(18.58, 18.63),
              time=("2018-01", "2018-01"),
              resolution=(-30, 30),
              group_by="solar_day"
             )

print(ds)

Plot the cloud masked landsat data:

In [ ]:
rgb(ds, index=0)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://digital-earth-africa.readthedocs.io/en/latest/genindex.html)